In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install bitsandbytes # 모델을 경량화해서 로드
!pip install transformers
!pip install peft # LoRA 튜닝을 하게 해주는 라이브러리
!pip install accelerate
!pip install trl
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━

In [3]:
from transformers import AutoTokenizer, LlamaForCausalLM, TrainingArguments, TrainerCallback
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
import peft
from trl import SFTTrainer
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

In [4]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained('beomi/llama-2-ko-7b', quantization_config=bnb_config, device_map={'': 0})
# fine-tuning으로 사용할 foundation model의 tokenizer 사용
tokenizer = AutoTokenizer.from_pretrained('beomi/llama-2-ko-7b')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/919M [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00003-of-00015.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(46336, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [6]:
tokenizer

LlamaTokenizerFast(name_or_path='beomi/llama-2-ko-7b', vocab_size=46336, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46331: AddedToken("<|sep|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46332: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46333: AddedToken("<|acc|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46334: A

In [7]:
train_df = pd.read_csv('/content/drive/MyDrive/데이콘/train.csv')
train_df

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,건축구조,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,TRAIN_002,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,건축구조,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,TRAIN_003,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,건축구조,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,TRAIN_004,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,건축구조,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...
...,...,...,...,...,...,...,...,...,...
639,TRAIN_639,벽장 부위 결로의 원인이 뭐야?,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,타 마감하자,벽장 부위 결로의 원인은 난방이 이웃한 방과동일한 조건이 되나 그 방에 비해 저온인...,"벽장 부위 결로의 원인은 주로 난방이 자리잡은 방이 내부 온도가 낮은 반면, 외부 ...",벽장 부위 결로가 발생하는 원인은 난방이 잘 이루어지지 않아 해당 공간이 저온인 반...,"벽장 부위 결로가 발생하는 원인은 난방이 인접한 방과 같은 조건을 갖추고 있지만, ...",벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
640,TRAIN_640,"AD, PD에 면한 벽체 결로의 원인이 뭐야?",벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,타 마감하자,"AD, PD에 면한 벽체 결로의 원인은 외기에 접하는 면 좌, 우측 벽체에는 단열재...","AD, PD에 면한 벽체 결로의 원인은 외기에 접하는 면 좌, 우측 벽체에는 단열재...","AD, PD에 면한 벽체 결로의 주된 원인은 외기에 접하는 면 좌, 우측 벽체에 단...","AD, PD에 면한 벽체 결로의 원인으로는 외기에 접하는 면 좌, 우측 벽체는 일반...","AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
641,TRAIN_641,외벽 모서리 부위에 결로가 발생하는 원인이 뭐야?,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,타 마감하자,외벽모서리 부위에 결로가 발생하는 원인은 높은 온도차 때문입니다. 외벽 모서리의 경...,외벽 모서리 부위에 결로가 발생하는 원인은 중요한 역할을 합니다. 외부에 비치는 세...,외벽 모서리 부위에 결로가 발생하는 원인으로는 높은 온도차가 대표적입니다. 외벽 모...,외벽 모서리 부위에 결로가 발생하는 원인은 실내와 외부의 온도차 때문입니다. 외벽 ...,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
642,TRAIN_642,창호 결로의 대책은 뭐야?,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,타 마감하자,창호결로의 대책은 제품을 선정할 때 KS에 규정된 프레임을 선정하고 유리의 열관류율...,창호 결로의 대책은 KS에 규정된 프레임을 선정하고 유리의 열관류율 및 결로 발생 ...,"창호 결로를 예방하기 위한 대책은 KS에 규정된 프레임을 선택하고, 열관류율이 적은...","창호 결로의 대책은 여러 가지가 있습니다. 먼저, 창호를 선택할 때 KS에 규정된 ...","창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


In [8]:
pairs = []
for i in range(1, 6):  # '답변_1'부터 '답변_5'까지
    # Check if the answer column exists in the data
    if f'답변_{i}' in train_df.columns:
        # Pair 질문_1 with 답변_i
        pair_1 = train_df[['질문_1', f'답변_{i}']].rename(columns={'질문_1': '질문', f'답변_{i}': '답변'})
        pairs.append(pair_1)

        # Pair 질문_2 with 답변_i
        pair_2 = train_df[['질문_2', f'답변_{i}']].rename(columns={'질문_2': '질문', f'답변_{i}': '답변'})
        pairs.append(pair_2)

# Combine all pairs into a single dataframe
train = pd.concat(pairs, ignore_index=True)

train

,질문,답변
0,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다."
2,철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...
3,철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ..."
4,철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.
...,...,...
6435,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
6436,벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,"AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
6437,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
6438,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,"창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


In [9]:
dataset = []
for question, answer in tqdm(zip(train['질문'], train['답변']), total = len(train)):
    dataset.append(f'<s>### Instruction: \n{question} \n\n### Response: \n{answer}</s>')
dataset = Dataset.from_dict({"text": dataset})
dataset = dataset.shuffle()
dataset

  0%|          | 0/6440 [00:00<?, ?it/s]

Dataset({
    features: ['text'],
    num_rows: 6440
})

In [10]:
dataset['text'][0]

'<s>### Instruction: \n도배지에 반점이 생기는 이유가 무엇인가요? \n\n### Response: \n도배지에 반점이 생기는 문제는 일반적으로 석고보드에 함유된 안료나 염료 등의 구성 물질이 도배지에 전이되어 발생한다고 알려져 있습니다. 이러한 경우, 해결 방법은 다음과 같습니다.  1. 반점을 바인더나 수성 프라이머로 코팅하고 다시 도배하는 방법 - 장점: 상대적으로 간단한 해결책으로 바로 수정이 가능합니다. - 단점: 재발 가능성이 있으며, 코팅 때문에 냄새가 발생할 수 있습니다.  2. 반점이 생긴 부분의 석고보드를 부분적으로 잘라내고 보강 후 재작업하는 방법 - 장점: 수정이 발생한 위치에서의 재발 위험이 낮고, 전체 석고보드 작업하는 것에 비해 시간과 비용이 적게 듭니다. - 단점: 수정하지 않은 부분에서 다시 반점이 생길 우려가 있을 수 있습니다.  3. 벽면 전체 석고보드를 잘라내고, 석고보드를 교체하여 재작업하는 방법 - 장점: 가장 근본적인 해결 방법으로 반점이 재발할 가능성이 낮아집니다. - 단점: 시간과 비용이 많이 소요되며, 교체 작업 중에는 먼지가 발생할 수 있습니다.  이런 문제를 해결하기 위해서는 상황에 맞는 적절한 방법을 선택하여 해결하는 것이 중요합니다.</s>'

In [11]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [12]:
lora_config_dict = {
    'lora_r' : 32,
    'lora_alpha' : 16,
    'lora_dropout' : 0.1,
    'lora_bias' : "none",
    'lora_task_type' : "CAUSAL_LM",
#     'target_modules': ["q_proj", "up_proj", "o_proj", "k_proj", "down_proj", "gate_proj", "v_proj"]
}



peft_config = peft.LoraConfig(
            lora_alpha=lora_config_dict['lora_alpha'],
            lora_dropout=lora_config_dict['lora_dropout'],
            r=lora_config_dict['lora_r'],
            bias=lora_config_dict['lora_bias'],
            task_type=lora_config_dict['lora_task_type'],
#             target_modules = lora_config_dict['target_modules']
        )

In [13]:
output_dir = '/content/drive/MyDrive/데이콘/llama2_train/llama2_lora_train'
# output_dir = 'llama2_lora_train'

In [14]:
train_param = {
    'num_train_epochs': 5,
    'per_device_train_batch_size': 4,
    'per_device_eval_batch_size': 4,
    'gradient_accumulation_steps': 1,
    'optim': "adamw_torch",
    'save_steps': 1000,
    'logging_steps': 10,
    'learning_rate': 0.0002,
    'weight_decay':  0.01,
    'max_grad_norm': 1,
    'warmup_ratio': 0.06,
    'group_by_length' : False,
    'lr_scheduler_type': 'cosine'
}

training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=train_param['num_train_epochs'],
            per_device_train_batch_size=train_param['per_device_train_batch_size'],
            per_device_eval_batch_size=train_param['per_device_eval_batch_size'],
            gradient_accumulation_steps=train_param['gradient_accumulation_steps'],
            optim=train_param['optim'],
            save_steps=train_param['save_steps'],
            logging_steps=train_param['logging_steps'],
            learning_rate=train_param['learning_rate'],
            weight_decay=train_param['weight_decay'],
            # fp16=kwargs['fp16'],
            # bf16=kwargs['bf16'],
            max_grad_norm=train_param['max_grad_norm'],
            # max_steps=kwargs['max_steps'],
            warmup_ratio=train_param['warmup_ratio'],
            group_by_length=train_param['group_by_length'],
            lr_scheduler_type=train_param['lr_scheduler_type'],
#             report_to="tensorboard"
        )

In [15]:
trainer = SFTTrainer(
            model=model,
            train_dataset=dataset,
            peft_config=peft_config,
            dataset_text_field="text",
            max_seq_length=None,
            tokenizer=tokenizer,
            args=training_args,
            packing=False,
            # neftune_noise_alpha=5,
            # callbacks=[AbortCallback()]
            )

Map:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [16]:
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)
# trainer.train(resume_from_checkpoint=True)

Step,Training Loss
10,3.292100
20,3.378900
30,3.386600
40,3.411600
50,3.208300
60,2.928400
70,2.650200
80,2.434900
90,2.046100
100,1.832700


TrainOutput(global_step=8050, training_loss=0.7723728016444615, metrics={'train_runtime': 14733.1602, 'train_samples_per_second': 2.186, 'train_steps_per_second': 0.546, 'total_flos': 2.0662072231152845e+17, 'train_loss': 0.7723728016444615, 'epoch': 5.0})

In [17]:
trainer.model.save_pretrained(output_dir)